<a href="https://colab.research.google.com/github/adamschmideg/ethereum/blob/master/travis/Visualize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl --remote-name https://raw.githubusercontent.com/adamschmideg/ethereum/master/travis/data/builds.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 42890  100 42890    0     0   263k      0 --:--:-- --:--:-- --:--:--  261k
